# PhageHostPredict: data processing

In this notebook, all the necessary data processing steps are completed in order to run PhageHostPredict models with RBPs and loci. The notebook transforms collections of phage genomes and bacterial genomes to collections of RBPs and loci.

## Libraries & files

In [1]:
import re
import os
import json
import math
import subprocess
import numpy as np
import pandas as pd
import processing_utils as pu
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from tqdm.notebook import tqdm
from os import listdir
from xgboost import XGBClassifier

In [2]:
valencia_dir = '/Users/dimi/GoogleDrive/PhD/4_PHAGEHOST_LEARNING/42_DATA/Valencia_data' # general directory
data_suffix = 'Valencia' # choose a suffix for the created data files

## Phage genome processing

In the block below, set the correct directory and location for Phanotate. The phage genomes directory should be a folder with all genomes present in them.

In [ ]:
phage_genomes_dir = valencia_dir+'/phages_genomes'
phanotate_loc = '/opt/homebrew/Caskroom/miniforge/base/envs/ML1/bin/phanotate.py'

#### Phanotate

In [ ]:
phage_files = listdir(phage_genomes_dir)
phage_files.remove('.DS_Store')
#record = SeqIO.read(phages_dir+'/'+phage_files[0], 'fasta')
bar = tqdm(total=len(phage_files), position=0, leave=True)
name_list = []; gene_list = []

for file in phage_files:
    # access PHANOTATE
    file_dir = phage_genomes_dir+'/'+file
    raw_str = phanotate_loc + ' ' + file_dir
    process = subprocess.Popen(raw_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = process.communicate()
    std_splits = stdout.split(sep=b'\n')
    std_splits = std_splits[2:] #std_splits.pop(0)
    
    # Save and reload TSV
    temp_tab = open(valencia_dir+'/phage_results.tsv', 'wb')
    for split in std_splits:
        split = split.replace(b',', b'') # replace commas for pandas compatibility
        temp_tab.write(split + b'\n')
    temp_tab.close()
    results_orfs = pd.read_csv(valencia_dir+'/phage_results.tsv', sep='\t', lineterminator='\n', index_col=False)
    
    # fill up lists accordingly
    name = file.split('_')[0]
    sequence = str(SeqIO.read(file_dir, 'fasta').seq)
    for j, strand in enumerate(results_orfs['FRAME']):
        start = results_orfs['#START'][j]
        stop = results_orfs['STOP'][j]
        
        if strand == '+':
            gene = sequence[start-1:stop]
        else:
            sequence_part = sequence[stop-1:start]
            gene = str(Seq(sequence_part).reverse_complement())
            
        name_list.append(name)
        gene_list.append(gene)
        
    # update progress
    bar.update(1)
bar.close()
os.remove(valencia_dir+'phage_results.tsv')

In [ ]:
# Export final genes database
genebase = pd.DataFrame(list(zip(name_list, gene_list)), columns=['phage_ID', 'gene_sequence'])
genebase.to_csv(valencia_dir+'/phage_genes'+data_suffix+'.csv', index=False)

#### RBP detection

In the code block below, provide the correct paths to the Pfam file, the trained XGB model, the saved phage genes, and protein embeddings of them.

In [14]:
pfam_file = '/Users/dimi/Documents/GitHub/PhageHostLearning/processing/RBPdetect_phageRBPs.hmm'
xgb_file = 'RBPdetect_xgb_model.json'
genebase = pd.read_csv(valencia_dir+'/phage_genes'+data_suffix+'.csv')
hmmer_path = '/Users/Dimi/hmmer-3.3.1' # e.g. '/Users/Sally/hmmer-3.3.1'
embeddings_file = ... # e.g. 'data/embeddings.csv'

In [15]:
# optionally press database first if not done already
#output, err = pu.hmmpress_python(hmmer_path, pfam_file)

In [18]:
# domain-based RBP detection
N_blocks = ['Phage_T7_tail', 'Tail_spike_N', 'Prophage_tail', 'BppU_N', 'Mtd_N', 
           'Head_binding', 'DUF3751', 'End_N_terminal', 'phage_tail_N', 'Prophage_tailD1', 
           'DUF2163', 'Phage_fiber_2', 'unknown_N0', 'unknown_N1', 'unknown_N2', 'unknown_N3', 'unknown_N4', 
            'unknown_N6', 'unknown_N10', 'unknown_N11', 'unknown_N12', 'unknown_N13', 'unknown_N17', 'unknown_N19', 
            'unknown_N23', 'unknown_N24', 'unknown_N26','unknown_N29', 'unknown_N36', 'unknown_N45', 'unknown_N48', 
            'unknown_N49', 'unknown_N53', 'unknown_N57', 'unknown_N60', 'unknown_N61', 'unknown_N65', 'unknown_N73', 
            'unknown_N82', 'unknown_N83', 'unknown_N101', 'unknown_N114', 'unknown_N119', 'unknown_N122', 
            'unknown_N163', 'unknown_N174', 'unknown_N192', 'unknown_N200', 'unknown_N206', 'unknown_N208']
C_blocks = ['Lipase_GDSL_2', 'Pectate_lyase_3', 'gp37_C', 'Beta_helix', 'Gp58', 'End_beta_propel', 
            'End_tail_spike', 'End_beta_barrel', 'PhageP22-tail', 'Phage_spike_2', 
            'gp12-short_mid', 'Collar', 
            'unknown_C2', 'unknown_C3', 'unknown_C8', 'unknown_C15', 'unknown_C35', 'unknown_C54', 'unknown_C76', 
            'unknown_C100', 'unknown_C105', 'unknown_C112', 'unknown_C123', 'unknown_C179', 'unknown_C201', 
            'unknown_C203', 'unknown_C228', 'unknown_C234', 'unknown_C242', 'unknown_C258', 'unknown_C262', 
            'unknown_C267', 'unknown_C268', 'unknown_C274', 'unknown_C286', 'unknown_C292', 'unknown_C294', 
            'Peptidase_S74', 'Phage_fiber_C', 'S_tail_recep_bd', 'CBM_4_9', 'DUF1983', 'DUF3672']

phage_genes_list = genebase['gene_sequence']
phage_genes_ids = genebase['phage_ID']

domain_based_detections = pu.RBPdetect_domains(hmmer_path, pfam_file, phage_genes_list, phage_genes_ids, 
                                               N_blocks=N_blocks, C_blocks=C_blocks, detect_others=False)

domain_preds = []
for pid in phage_genes_ids:
    if pid in list(domain_based_detections['identifier']):
        domain_preds.append(1)
    else:
        domain_preds.append(0)

Scanning the genes: 100%|███████████████████| 5657/5657 [01:35<00:00, 59.33it/s]


In [19]:
domain_based_detections

,identifier,DNASeq,N_blocks,C_blocks,N_ranges,C_ranges
0,K39PH122C2,ATGGCATTCAGCTGGCAAGAGTCGGTAAAGCCTGCAGGTACTCAGG...,"[Phage_T7_tail, unknown_N2]","[unknown_C76, unknown_C234, unknown_C286]","[(12, 140), (3, 105)]","[(140, 791), (142, 263), (26, 37)]"
1,K39PH122C2,ATGGCTTTAGTAAAACTTACACGGGTAGCGGAGTGGCTCGGAACGT...,[],"[Pectate_lyase_3, Peptidase_S74]",[],"[(38, 126), (543, 603)]"
2,K60PH164C1,ATGAATTCTCATAACCCGTTTAACACAGGTGGTAGTGGGTGCACCA...,[unknown_N82],[unknown_C105],"[(124, 237)]","[(235, 331)]"
3,K45PH128C2,ATGGCTGATACAACACAATTTGAACAGGCTGTGGATCAGGTTATCG...,[],[unknown_C123],[],"[(66, 115)]"
4,K45PH128C2,ATGGCAATTCCTACTATTCCTCTTCAGATATGGGCTGAAAGTGACG...,[unknown_N48],[],"[(0, 82)]",[]
...,...,...,...,...,...,...
89,K33PH14C2,ATGACTAATATCAAGGCCCGCAAGGGCGGTTCCAGCCAGCCGCGAA...,"[unknown_N12, unknown_N163, unknown_N206, unkn...",[],"[(0, 199), (3, 186), (2, 198), (3, 194), (9, 1...",[]
90,K33PH14C2,ATGGACGCAAACGGCGTTATCACCGGGACTGGTACGAAGTGGCAAT...,[unknown_N11],[],"[(0, 163)]",[]
91,K33PH14C2,CTGGATTCGTGGGCCAATGTTGATGGCGTAGGTTCCATGTACACAA...,[],[DUF1983],[],"[(2, 77)]"
92,K35PH164C3,ATGGATAATGAAATACGCACGGTGGTCACGTACAACCTAGATGGGG...,[Phage_T7_tail],[],"[(0, 117)]",[]


In [ ]:
# ML-based RBP detection
embeddings_df = pd.read_csv(embeddings_file)
embeddings = np.asarray(embeddings_df.iloc[:, 2:])

xgb_saved = XGBClassifier()
xgb_saved.load_model(xgb_file) # load trained model

score_xgb = xgb_saved.predict_proba(embeddings)[:,1]
preds_xgb = (score_xgb > 0.5)*1

In [ ]:
# take union or intersection of both detection methods
...

In [29]:
# make a dataframe and unique protein IDs
proteins = [str(Seq(sequence).translate())[:-1] for sequence in domain_based_detections['DNASeq']]
unique_ids = []
for item in domain_based_detections['identifier']:
    check = 0; count = 0
    while check == 0:
        unique = item+'_RBP'+str(count)
        if unique not in unique_ids:
            unique_ids.append(unique)
            check = 1
        else:
            count += 1
            
RBPs = pd.DataFrame({'phage_nr': domain_based_detections['identifier'], 
                      'host': ['klebsiella_pneumoniae']*len(domain_based_detections['identifier']), 
                      'host_accession': ['-']*len(domain_based_detections['identifier']), 
                      'DNASeq': domain_based_detections['DNASeq'], 
                      'ProteinSeq': proteins, 'N_blocks': domain_based_detections['N_blocks'], 
                      'C_blocks': domain_based_detections['C_blocks'], 'N_ranges': domain_based_detections['N_ranges'], 
                      'C_ranges': domain_based_detections['C_ranges'], 'unique_ID': unique_ids})

In [45]:
RBPs.head()

,phage_nr,host,host_accession,DNASeq,ProteinSeq,N_blocks,C_blocks,N_ranges,C_ranges,unique_ID
0,K39PH122C2,klebsiella_pneumoniae,-,ATGGCATTCAGCTGGCAAGAGTCGGTAAAGCCTGCAGGTACTCAGG...,MAFSWQESVKPAGTQDIQCDIEYLDKSYIHVYLDGAETTAFTWTSS...,"[Phage_T7_tail, unknown_N2]","[unknown_C76, unknown_C234, unknown_C286]","[(12, 140), (3, 105)]","[(140, 791), (142, 263), (26, 37)]",K39PH122C2_RBP0
1,K39PH122C2,klebsiella_pneumoniae,-,ATGGCTTTAGTAAAACTTACACGGGTAGCGGAGTGGCTCGGAACGT...,MALVKLTRVAEWLGTYIHSLSSVQRLLGSKLDDTLCVLDFGAVADY...,[],"[Pectate_lyase_3, Peptidase_S74]",[],"[(38, 126), (543, 603)]",K39PH122C2_RBP1
2,K60PH164C1,klebsiella_pneumoniae,-,ATGAATTCTCATAACCCGTTTAACACAGGTGGTAGTGGGTGCACCA...,MNSHNPFNTGGSGCTNDFRSADPLVDRIIGDAYHVVKEVYLALGNL...,[unknown_N82],[unknown_C105],"[(124, 237)]","[(235, 331)]",K60PH164C1_RBP0
3,K45PH128C2,klebsiella_pneumoniae,-,ATGGCTGATACAACACAATTTGAACAGGCTGTGGATCAGGTTATCG...,MADTTQFEQAVDQVIEDSERLHKVVNGSAIDTVIVEDGSTIPTLRK...,[],[unknown_C123],[],"[(66, 115)]",K45PH128C2_RBP0
4,K45PH128C2,klebsiella_pneumoniae,-,ATGGCAATTCCTACTATTCCTCTTCAGATATGGGCTGAAAGTGACG...,MAIPTIPLQIWAESDVVLPNAHTANKISPIADLWDKGWDLGEKPAC...,[unknown_N48],[],"[(0, 82)]",[],K45PH128C2_RBP1


#### Filter for length & save dataframe

In [37]:
to_delete = [i for i, protein_seq in enumerate(RBPs['ProteinSeq']) if (len(protein_seq)<200 or len(protein_seq)>1500)]
RBPs = RBPs.drop(to_delete)
RBPs = RBPs.reset_index(drop=True)

In [42]:
RBPs.to_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv', index=False)

#### CD-HIT for 3D structure predictions

Here, we cluster the identified RBPs to see how they cluster together. Subsequently, the representative of each of the clusters is subjected to AlphaFold's 3D structure prediction (on high-performance computing infrastructure).

Let's keep it simple: all the already known RBPs (with B-helix) that are representatives are white listed by default (and subsequently all the sequences in those clusters), and all other representatives are subjected to a 3D structure prediction for manual check of B-helix presence. The representatives are automatically generated in a fasta file as output of the clustering. The representatives can then be subjected to 3D structure prediction (manual step on HPC). If the B-helix is present, we write the protein name in the white list file (manually), and subsequently run the code under 'Interaction matrix construction' again to keep all the sequences in the corresponding clusters.

In [47]:
cdpath = '/Users/dimi/cd-hit-v4.8.1-2019-0228'
known_klebsiella_rbps = '/Users/dimi/GoogleDrive/PhD/4_PHAGEHOST_LEARNING/42_DATA/klebsiella_RBP_data/sequences/klebsiella_RBPs.fasta'

In [48]:
# adding known Klebsiella RBPs
RBPs = pd.read_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv')
combined_fasta = open(valencia_dir+'/RBPs_for_clustering'+data_suffix+'.fasta', 'w')
white_list = open(valencia_dir+'/RBPs_white_list'+data_suffix+'.txt', 'w')

for record in SeqIO.parse(known_klebsiella_rbps, 'fasta'):
    sequence = str(record.seq)
    rec_id = record.id
    combined_fasta.write('>identified_'+rec_id+'\n'+sequence+'\n')
    white_list.write('identified_'+rec_id+'\n')
for i, protein in enumerate(RBPs['ProteinSeq']):
    unique_id = RBPs['unique_ID'][i]
    combined_fasta.write('>'+unique_id+'\n'+protein+'\n')
combined_fasta.close()
white_list.close()

In [54]:
# cluster with CD-HIT
input_file = valencia_dir+'/RBPs_for_clustering'+data_suffix+'.fasta'
output_file = valencia_dir+'/RBPs_clusters'+data_suffix
cdout, cderr = pu.cdhit_python(cdpath, input_file, output_file)

## Bacterial genome processing

#### Set file paths

In [3]:
bacterial_genomes_dir = valencia_dir+'/klebsiella_genomes/fasta_files'
database_path = '/Users/dimi/GoogleDrive/PhD/4_PHAGEHOST_LEARNING/42_DATA/klebsiella_RBP_data/Klebsiella_k_locus_primary_reference.gbk'
kaptive_directory = '/Users/dimi/Documents/GitHub/PhageHostLearning/processing'

#### Construct dataframe (optional)

#### Run Kaptive

In [4]:
Locibase, seros = pu.compute_kaptive_from_directory(kaptive_directory, database_path, bacterial_genomes_dir, valencia_dir)
pd.DataFrame(seros, columns=['sero']).to_csv(valencia_dir+'/serotypes'+data_suffix+'.csv', index=False)
dict_file = open(valencia_dir+'/Locibase'+data_suffix+'.json', 'w')
json.dump(Locibase, dict_file)
dict_file.close()

100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [1:40:26<00:00, 97.21s/it]


In [5]:
dict_file = open(valencia_dir+'/Locibase'+data_suffix+'.json')
loci_dictionary = json.load(dict_file)

## Interaction matrix construction

In this section, we construct and filter (refine) the interaction matrix. The interaction matrix is the third and final piece of information we need in order to construct machine learning models later. Basically, the interaction matrix contains the labels, which are now displayed as a matrix as we are dealing with pairs of objects.

Practically, it seems most likely that the (initial) interaction matrix is constructed at the phage-bacterium level (not at the RBP-locus level). Therefore, the first step is that this matrix should be reworked to the level of RBPs-receptors. Afterwards, it can be filtered further based on the white listed RBPs that contain a B-helix (see above). The columns of the initial interaction matrix should correspond with the phage names in the constructed RBPbase earlier, which corresponds with the names of the fasta files in the phage genomes directory. Same for the bacterial names. In the end, we want the interaction matrix to correspond (in rows and columns) to the RBPbase and Locibase!

Below, the initial interaction matrix is an MS Excel file that was put together based on experimental results.

#### Construct interaction matrix from initial file (run again if new RBPs are detected, genomes are added or new RBPs are white listed)

In [3]:
#files = listdir(bacterial_genomes_dir)
#files.remove('.DS_Store')
#row_names = [file.split('_')[0] for file in files] # same as accessions in klebsiellaGenomesValencia
IM = pd.read_excel(valencia_dir+'/klebsiella_phage_host_interactions.xlsx', index_col=0, header=0)
IM_klebsiella_names = list(IM.index) # corresponds to bacteria accessions
IM_phage_names = list(IM.columns) # corresponds to phage_nr in RBPbase
klebsiellas = pd.read_csv(valencia_dir+'/klebsiellaGenomes'+data_suffix+'.csv')
RBPbase = pd.read_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv')

row_names = klebsiellas['accession']
column_names = RBPbase['unique_ID']
interactions = np.zeros((len(row_names), len(column_names)))
for i, row in enumerate(row_names):
    row_index = IM_klebsiella_names.index(row) # index in IM
    for j, col in enumerate(column_names):
        phage_name = RBPbase['phage_nr'][j] # link between phages and RBPs
        col_index = IM_phage_names.index(phage_name) # index in IM
        interactions[i,j] = IM.iloc[row_index, col_index]

interactions_df = pd.DataFrame(interactions, index=row_names, columns=column_names)
interactions_df.to_csv(valencia_dir+'/interactions'+data_suffix+'.csv')

In [4]:
interactions_df.shape

(62, 88)

#### Delete identical interactions (run again if new RBPs are detected, genomes are added or new RBPs are white listed)

In [5]:
pwjulia_directory = '/Users/dimi/Documents/GitHub/PhageHostLearning'
RBPbase = pd.read_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv')

In [19]:
# compute RBP similarities (run only if new RBPs are detected or genomes are added)
fasta_temp = open(valencia_dir+'/RBPbase'+data_suffix+'.fasta', 'w')
for i, sequence in enumerate(RBPbase['ProteinSeq']):
    name = RBPbase['unique_ID'][i]
    fasta_temp.write('>'+name+'\n'+sequence+'\n')
fasta_temp.close()
pu.pairwise_alignment_julia(valencia_dir+'/RBPbase'+data_suffix+'.fasta', 'protein', pwjulia_directory)

(b'\rProgress:   0%|                                         |  ETA: 0:28:17\x1b[K\rProgress:  26%|\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x8b                              |  ETA: 0:00:06\x1b[K\rProgress:  30%|\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x8d                            |  ETA: 0:00:05\x1b[K\rProgress:  33%|\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x8b                           |  ETA: 0:00:04\x1b[K\rProgress:  36%|\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88                          |  ETA: 0:00:04\x1b[K\rProgress:  40%|\x

In [9]:
# compute loci similarities (run only if new RBPs are detected or genomes are added)
pu.pairwise_alignment_julia(valencia_dir+'/kaptive_results_all_loci.fasta', 'DNA', pwjulia_directory)

(b'\rProgress:   0%|                                         |  ETA: 1:46:46\x1b[K\rProgress:   0%|\xe2\x96\x8f                                        |  ETA: 1:25:14\x1b[K\rProgress:   1%|\xe2\x96\x8e                                        |  ETA: 0:55:06\x1b[K\rProgress:   1%|\xe2\x96\x8d                                        |  ETA: 1:11:20\x1b[K\rProgress:   1%|\xe2\x96\x8d                                        |  ETA: 1:14:36\x1b[K\rProgress:   1%|\xe2\x96\x8d                                        |  ETA: 1:13:15\x1b[K\rProgress:   1%|\xe2\x96\x8d                                        |  ETA: 1:15:39\x1b[K\rProgress:   1%|\xe2\x96\x8c                                        |  ETA: 1:17:26\x1b[K\rProgress:   1%|\xe2\x96\x8c                                        |  ETA: 1:19:26\x1b[K\rProgress:   1%|\xe2\x96\x8c                                        |  ETA: 1:20:54\x1b[K\rProgress:   1%|\xe2\x96\x8c                                        |  ETA: 1:19:20\x1b[K\rProgress:   1%|\

In [6]:
# adjust interaction matrix
loci_sim = np.loadtxt(valencia_dir+'/kaptive_results_all_loci.fasta_score_matrix.txt')
rbp_sim = np.loadtxt(valencia_dir+'/RBPbaseValencia.fasta_score_matrix.txt')
IM = pd.read_csv(valencia_dir+'/interactions'+data_suffix+'.csv', index_col=0)
interactions = np.asarray(IM)

# delete identical interactions
for i in range(rbp_sim.shape[0]-1):
    for j in range(i, rbp_sim.shape[0]):
        if (rbp_sim[i,j] == 1) and (i != j):
            # get the corresponding loci
            locus_i = list(np.where(interactions[:, i] == 1)[0])
            locus_j = list(np.where(interactions[:, j] == 1)[0])
            
            for li in locus_i:
                for lj in locus_j:
                    if loci_sim[li, lj] == 1: # if identical delete one of both
                        interactions[lj,j] = np.nan

# save new interaction matrix
row_names = IM.index; column_names = IM.columns
IM_no_identicals = pd.DataFrame(interactions, index=row_names, columns=column_names)
IM_no_identicals.to_csv(valencia_dir+'/interactions'+data_suffix+'.csv')

#### Filter for white listed RBPs (run again if new RBPs are detected, genomes are added or new RBPs are white listed)

In [7]:
# loop over clusters: if one in RBPs_with_Bhelix -> white list entire cluster
white_list = open(valencia_dir+'/RBPs_whitelist'+data_suffix+'.txt')
white_list = white_list.read().split('\n')
output_file = valencia_dir+'/RBPs_clusters'+data_suffix
clusters = open(output_file+'.clstr')

cluster_iter = 0
white_check = 0
cluster_accessions = []
sequences_to_keep = []
for line in clusters.readlines():
    # new cluster
    if line[0] == '>':
        # finish old cluster if not first one
        if (cluster_iter > 0) and (len(cluster_accessions) >= 1):
            if white_check == 1: # an identified RBP in cluster
                sequences_to_keep = sequences_to_keep+cluster_accessions
        # initiate new cluster
        cluster_accessions = []
        cluster_iter += 1
        white_check = 0
    # in a cluster
    else:
        acc = line.split('>')[1].split('...')[0]
        cluster_accessions.append(acc)
        if acc in white_list:
            white_check = 1
        
# finish last cluster
if len(cluster_accessions) >= 1:
    if white_check == 1: # an identified RBP in cluster
        sequences_to_keep = sequences_to_keep+cluster_accessions
print('number of sequences kept: ', len(sequences_to_keep))

number of sequences kept:  184


In [8]:
# filter interaction matrix based on white listed RBPs
IM = pd.read_csv(valencia_dir+'/interactions'+data_suffix+'.csv', index_col=0)
interactions = np.asarray(IM)
for j, column in enumerate(column_names):
    if column not in sequences_to_keep:
        interactions[:,j] = np.nan

row_names = IM.index; column_names = IM.columns
interactions_whitelist_only = pd.DataFrame(interactions, index=row_names, columns=column_names)
interactions_whitelist_only.to_csv(valencia_dir+'/interactions'+data_suffix+'.csv')

In [11]:
np.sum(np.sum(interactions_whitelist_only == 1))

100

In [10]:
# coverage
(np.sum(np.sum(interactions_whitelist_only == 0)) + np.sum(np.sum(interactions_whitelist_only == 1))) / (interactions_whitelist_only.shape[0]*interactions_whitelist_only.shape[1])

0.6354472140762464

#### filter for mono-RBP phages (run again if new RBPs are detected, genomes are added or new RBPs are white listed)

In [12]:
IM = pd.read_csv(valencia_dir+'/interactions'+data_suffix+'.csv', index_col=0)
interactions = np.asarray(IM)
multi_RBP_phages = [name[:-5] for name in IM.columns if 'RBP1' in name]
print(multi_RBP_phages)
for j, column in enumerate(IM.columns):
    if column[:-5] in multi_RBP_phages:
        for i in range(IM.shape[0]):
            interactions[i,j] = np.nan
            
row_names = IM.index; column_names = IM.columns
IM_mono = pd.DataFrame(interactions, index=row_names, columns=column_names)
IM_mono.to_csv(valencia_dir+'/interactions_mono'+data_suffix+'.csv')

['K39PH122C2', 'K45PH128C2', 'K17alfa61', 'K54lambda2', 'K54lambda1', 'K51PH129C1', 'K50PH164C1', 'K18PH07C1', 'K14PH164C1', 'K28PH129', 'K13PH07C1S', 'K62PH164C2', 'K2alfa62', 'K63PH128', 'K8PH128', 'K21lambda1', 'K2PH164C1', 'K34PH164', 'K65PH164', 'K52PH129C1', 'K29PH164C1', 'K23PH08C2', 'K46PH129', 'K53PH164C2', 'K2PH164C2', 'K9PH25C2', 'K69PH164C2', 'K6PH25C3', 'K13PH07C1L', 'K33PH14C2']


In [15]:
np.sum(np.sum(IM_mono == 1))

29

In [14]:
# coverage
(np.sum(np.sum(IM_mono == 0)) + np.sum(np.sum(IM_mono == 1))) / (IM_mono.shape[0]*IM_mono.shape[1])

0.22727272727272727